In [35]:
import spacy
from spacy.tokens import DocBin

from spacy.training import Example, offsets_to_biluo_tags
from spacy.training.corpus import Corpus
from spacy.training.loop import train
from spacy.util import load_config
from spacy.training import Example

from spacy import displacy

from sklearn.model_selection import train_test_split
import json

from spacy.scorer import Scorer

import random

In [36]:
# Função para converter JSON anotado
def convert_json_to_spacy_format(json_data):
    spacy_format_data = []

    for entry in json_data:
        text = entry['text']
        entities = entry['label']
        spacy_entities = []
        
        # Convertendo cada entidade para o formato spacy
        for entity in entities:
            start, end, label = entity
            spacy_entities.append((start, end, label))
        
        # Adicionando ao formato do spaCy
        spacy_format_data.append((text,spacy_entities))
    
    return spacy_format_data

In [ ]:
with open(f'data\corpus_annotated\henrique.jsonl', 'r', encoding='utf-8') as f:
    json_data = [json.loads(line) for line in f]

json_data

In [ ]:
# Convertendo o JSON para Spacy
data = convert_json_to_spacy_format(json_data)

data

In [39]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(f'Tamanho do treino: {len(train_data)}')
print(f'Tamanho do teste: {len(test_data)}')

In [41]:
nlp = spacy.blank("pt")  # Modelo vazio em português
ner = nlp.add_pipe("ner")

##### Treinamento do modelo

In [ ]:
optimizer = nlp.initialize()

for i in range(100):  # Número de épocas
    random.shuffle(train_data)
    print(f'Época: {str(i)}')
    for raw_text, entity_offsets in train_data:
        # Atualizando o modelo com os textos e entidades
        doc = nlp.make_doc(raw_text)
        entities = {"entities": entity_offsets}
        example =  Example.from_dict(doc, entities)
        nlp.update([example], sgd=optimizer, drop=0.25)

##### Avaliação do modelo

In [ ]:
examples = []
for raw_text, entity_offsets in test_data:
    doc = nlp.make_doc(raw_text)
    entities = {"entities": entity_offsets}
    example = Example.from_dict(doc, entities)
    examples.append(example)

In [44]:
# Avaliação do modelo
scores = nlp.evaluate(examples)

In [ ]:
print("Chaves disponíveis no dicionário de scores:", scores.keys())

In [ ]:
# Acessar a precisão geral
print("Precisão geral:", scores["ents_p"])
print("Recall:", scores["ents_r"])
print("F1 Score:", scores["ents_f"])

# Acessar métricas específicas para cada tipo de entidade
for ent_type, metrics in scores["ents_per_type"].items():
    print(f"\nEntidade: {ent_type}")
    print(f" - Precision: {metrics['p']}")
    print(f" - Recall: {metrics['r']}")
    print(f" - F1 Score: {metrics['f']}")


In [47]:
texts = [
    """153: O melhor ataque da Copa São Paulo de Juniores enfrenta hoje às 20h no Pacaembu a melhor defesa da competição. O São Paulo, que já marcou 22 gols no torneio, joga contra o Vitória-BA, time que sofreu apenas um gol no campeonato. A partida vale pelas quartas-de-final e quem perder está fora da Copa São Paulo.
Embora seja considerado o favorito, o São Paulo prefere "respeitar o adversário". Para o preparador físico da equipe, Ademir de Souza Filho, "nesse torneio não há favoritos, pois as equipes estão niveladas". Para ele, "O Vitória é um time rápido, que joga e deixa jogar. Vai ser um bom jogo".
O treinador do São Paulo, Murici, aponta a boa formação do Vitória como responsável pelo sucesso da equipe baiana. "Eles têm um bom conjunto, é um time bem organizado, mas nós vamos tentar furar o bloqueio deles", afirmou.
O São Paulo entra com seu time titular para enfrentar a equipe baiana. O técnico Murici vai poder contar com a volta do volante Mona, que não jogou contra o Caxias-RS na terça-feira, pois tinha dois cartões amarelos e foi poupado. O zagueiro Nem, suspenso com o terceiro amarelo na última partida, também retorna à equipe.
No Vitória, as contusões são o maior problema para o técnico Péricles Chamusca escalar sua equipe hoje. O zagueiro Flávio sofreu uma contratura na coxa direita e dificilmente joga. Edu, que teve um entorse no tornozelo mas se recuperou, deve entrar em seu lugar. Na lateral-direita, Dedimar teve um traumatismo na perna direita no jogo contra o Corinthians e deve ser substituído pelo ponta Antônio Marcos. Além disso, o atacante Careca, com um entorse no joelho esquerdo, também é dúvida.
Para o treinador baiano, o São Paulo é o favorito da partida. "É um time praticamente imbatível", disse ele. "Mas nós vamos lutar muito para sermos a 'zebra' da rodada", completou. Segundo Chamusca, o fato de o Vitória ter a melhor defesa do torneio não significa que a equipe vá jogar retrancada. "Num jogo eliminatório, a retranca é um mau negócio. Vamos tentar jogar de igual para igual", afirmou.""",

    """129: Meus amigos, meus inimigos, não deu para entender por que a diretoria do Corinthians não acertou a compra do Rivaldo no momento em que ele começou a mostrar o seu potencial.
Ou por que não fixaram o preço do seu passe desde o início. Deixaram para negociar depois do sucesso dele no campeonato e na seleção e o resultado está aí.
Bem mais esperta foi a diretoria do São Paulo com o Juninho. O Corinthians apenas serviu de marketing nacional para o Mogi Mirim.
Animais e times
Já se sabe que os americanos não são pessoas muito sérias. No mundo do esportes, então, as chances da Gringolândia do tio Dave se tornar um lugar muito parecido com o reino das bonecas Barbie são bastante grandes.
Lá, como aqui, os times –seja de basquete ou de futebol– também são conhecidos por apelidos. Falamos no Peixe, no Leão, no Touro do Vale etc.
Mas, por mais fanáticos que possam ser os torcedores brasileiros pelos seus símbolos, penso que ainda não chegamos a tantas intimidades com o mundo da fantasia como naquele país –onde, aliás, um grupo de feministas passou a considerar que todo ato sexual humano que não tenha a reprodução como finalidade deve ser considerado um estupro.
Um professor da Universidade Xavier, Ohio, chamado Ernest Fontana –segundo Bob Matthews, do jornal "USA Today"–, está lançando o Grande Debate do Apelido dos Times Universitários.
Na listinha do ocupado professor Matthews, Tigre é o apelido adotado por 25 times de universidades americanas. Certamente pelo aspecto agressivo: os tigres da ira são preferíveis aos camelos da cultura, como diria o poeta. A América, realmente, não é um tigre de papel.
Em segundo lugar na lista de animais preferidos como símbolo, está o Buldogue, talvez por ser metade cachorro metade touro. Ele foi adotado por 22 universidades. Em terceiro neste curioso ranking, fica a águia, com 19 adoções. Como se vê, também por lá há quem prefira adotar a frieza e argúcia da ave de rapina.
Bob Matthews diz que ainda há uma lista grande de animais disponíveis, que não encontraram times para adotá-los. E sugere Girafa para algum time de basquete ou Gafanhoto (talvez pela referência à grama que há no nome do inseto em inglês) para um time de futebol.
Mas, nada bate o apelido adotado pelo time da Universidade da Califórinia, em Santa Cruz. Seus torcedores adoram ser chamados de alguma coisa parecida como os Lesmas de Banana.
Pode parecer engraçado para nós brasileiros. Mas, como tudo na vida, depende da perspectiva de quem está olhando. O colunista Matthews, por exemplo, inclui o Porco como um dos animais não recomendados para ser símbolo de time algum.
E todos nós conhecemos uma imensa torcida que, em caso raríssimo de identificação com os inimigos na história do esporte, adotou o símbolo da difamação como o apelido da afirmação de sua identidade –até então bastante maltratada pelos adversários.
Que tipo de estudos poderia desenvolver o expert no assunto, o nosso professor Fontana, quando descobrir que abaixo do Equador milhares de pessoas se juntam em estádios para gritarem "Pooorco" orgulhosamente?
Segundo o professor Fontana, os times devem mesmo procurar seus apelidos nos animais. "Ursos não podem falar", justifica ele.
Mas, no país das feministas furiosas e do Politicamente Correto, espere até que Associação dos Protetores do Animais comece a falar por eles.
Matinas Suzuki Jr. escreve nesta coluna às terças, quintas e sábados.""",

    """165: Mesmo sem jogar bem, o São Paulo venceu o Santos por 2 a 0, gols de Juninho e Valdeir, ontem no Morumbi, e já é líder isolado do Paulistão, com 6 pontos ganhos. Jogando sob chuva, que chegou a ameaçar a realização da partida, a equipe são-paulina foi sobrepujada taticamente pelo adversário, mas venceu graças à eficiência de seus contra-ataques e à má atuação do goleiro Gilberto.
No duelo entre os "renegados" Dinho e Axel, venceu o santista. "Me sinto mais à vontade para jogar no Santos que no São Paulo", reconheceu. Pepe utilizou o volante para anular Leonardo e subir ao ataque. Por sua vez, Axel ficou perdido no rápido meio-campo do São Paulo: não sabia se marcava ou subia à frente. "Acho que foi uma boa estréia. Ganhamos e é isto que interessa", declarou.
Não foi só o novo volante que sentiu falta de orientação. Com uma burocrática atuação do interino Muricy, ao São Paulo faltou uma mudança tática para liberar Leonardo ou encontrar um posicionamento mais adequado a Axel. Muricy não comprometeu, mas deixou a impressão de que o time sente falta de Telê.
Juninho teve que correr muito para compensar isso. Com Leonardo anulado por Dinho, o meia ficava ou preso à marcação ou abandonado no ataque, assim como Valdeir. Graças a sua habilidade, conseguiu chegar ao gol santista mais de uma vez. E, aos 38min do primeiro tempo, num lance de sorte, contou com a falha homérica de Gilberto, que não segurou um chute fraco de fora da área. "A equipe ainda sente a falta de entrosamento e de preparo físico", afirmou.
No segundo tempo, com o campo mais seco, esperava-se que a eficiência do São Paulo anulasse a garra santista. Mas as indefinições táticas continuaram e o inverso aconteceu. Macedo e Paulinho Kobayshi infernizaram, sem resultado, a defesa são-paulina, fragilizada pela falta de entrosamento entre Válber e Júnior Baiano. Para fechar o jogo, Valdeir fez um belo gol aos 38 min, enganando Gilberto ao chutar com o pé direito. Uma jogada trabalhada com Leonardo, momento raro em que a dupla funcionou.""",

    """172: A atuação do pivô Hakeem Olajuwon foi fundamental na vitória do Houston Rockets sobre o Utah Jazz por 106 a 101, anteontem, na NBA, quebrando uma série de quatro derrotas consecutivas de sua equipe. Olajuwon marcou 40 pontos e pegou dez rebotes. Com a vitória, o Houston abriu vantagem de três vitórias sobre o Jazz na luta pelo título da Divisão do Meio-Oeste. O San Antonio Spurs assumiu o segundo lugar do grupo, ao vencer o Dallas Mavericks por 103 a 83.
O New Jersey Nets humilhou o Golden State Warriors em East Rutherford (124 a 98), com grande desempenho de Kenny Anderson, responsável por 20 pontos, dez assistências e dez bols roubadas.
O Seattle Supersonics perdeu uma invencibilidade de quatro jogos ao cair diante do Philadelphia 76ers, em casa, por 95 a 88. O arremesso de Andrew Toney, que fez 27 pontos, foi o responsável pela derrota dos sonics. No entanto, Toney não foi o único herói da noite. O pivô Moses Malone teve papel importante, anotando 22 pontos e pegando 13 rebotes.
O que não é fácil para o Dallas Mavericks é saber quando seus astros estão em uma noite inspirada. Ontem, no jogo contra o San Antonio Spurs, a equipe não teve a mínima chance. Apesar da boa atuação de sua linha de frente, que fez 54 pontos, e da boa atuação do armador Derek Harper, que fez 20 pontos, o Dallas ainda perdeu. O responsável pelo triunfo foi David Robinson, com 25 pontos e 15 rebotes, além de uma defesa excepcional.
A NBA ainda teve mais duas partidas: o Portland Trail Blazers venceu o Phoenix Suns por 98 a 85, enquanto o Miami Heat derrotou o Los Angeles Clippers por 107 a 83, com destaque para a atuação de Alonzo Mourning, que fez 23 pontos e pegou 12 rebotes.""",

    """163: O 5º Cebrap foi realizado em São Paulo nos dias 1º e 2 de junho, no Museu de Arte de São Paulo (MASP). O evento, que já se tornou tradicional, teve como tema 'As Mulheres e os Novos Cenários'. Ao todo, foram 33 mesas de discussão e oficinas e mais de 300 participantes de todo o Brasil.
Uma das mesas mais esperadas foi a que tratou do trabalho feminino nas redes sociais, com a presença de Renata Randel, Luiza Brasil e Ana Clara. Ao final da mesa, as participantes tiveram a oportunidade de discutir o que pode ser feito para melhorar a visibilidade das mulheres nesse ambiente.
A mesa também trouxe a participação do cantor e compositor Adriano Trindade, que falou sobre sua experiência e sobre a importância da representatividade nas redes sociais. "Precisamos ter mais mulheres em cargos de liderança nas redes sociais", disse.
O evento contou com diversas atividades culturais, como exposições de arte, apresentações musicais e performances de dança. "Queremos mostrar a força da mulher na arte", afirmou a organizadora do evento, Maria Teresa.
O 5º Cebrap é uma iniciativa da Associação Brasileira de Mulheres Artistas, que busca promover a igualdade de gênero nas artes e na cultura. "Precisamos garantir que as mulheres tenham um espaço de fala e protagonismo", finalizou Maria Teresa.""",

    """104: O Brasil recebeu um prêmio importante pela sua atuação na Conferência das Nações Unidas sobre Mudanças Climáticas (COP). O evento, realizado em Glasgow, teve como objetivo promover ações para combater as mudanças climáticas e preservar o meio ambiente.
O Brasil foi reconhecido por seu compromisso em reduzir as emissões de carbono e promover o desenvolvimento sustentável. "Estamos trabalhando para proteger nossas florestas e garantir um futuro mais verde para as próximas gerações", afirmou o Ministro do Meio Ambiente, Joaquim Leite.
Durante a conferência, foram apresentadas diversas iniciativas para enfrentar os desafios climáticos, como a preservação da Amazônia e o incentivo a energias renováveis. "É fundamental que todos os países se unam para combater as mudanças climáticas", disse Leite.
O evento também trouxe a participação de líderes mundiais e representantes de organizações não governamentais, que discutiram maneiras de promover a ação global em prol do meio ambiente. "Precisamos agir agora para garantir um futuro sustentável", enfatizou a ativista ambiental Greta Thunberg.
O Brasil se comprometeu a cumprir as metas estabelecidas no Acordo de Paris e a trabalhar em parceria com outros países para atingir os objetivos de desenvolvimento sustentável. "Juntos, podemos fazer a diferença", concluiu o Ministro Joaquim Leite.""",
]





In [ ]:
options = {
    "colors": {
        "time": "#009CE0",
        "goleiro": "#FCDC00",
        "defensor": "#FCC400",
        "meio-campista": "#F44E3B",
        "atacante": "#9F0500",
        "campeonato": "#0062B1",
        "estádio": "#7B64FF",
        "técnico": "#A4DD00",
        "árbitro principal": "#FA28FF",
        "assistente": "#FDA1FF",
        "placar": "#0C797D"
    },
    "template": """
        <mark class="entity" style="background: {bg}; padding: 0.45em 0.6em; margin: 0 0.25em;
        line-height: 1; border-radius: 0.35em; border: 1px solid #ddd">
            {text}<span style="font-size: 0.8em; font-weight: bold; line-height: 1; 
            padding-left: 0.5rem; color: #fff;">{label}</span>
        </mark>
    """
}

for text in texts:
    doc = nlp(text)
    print()
    displacy.render(doc, style="ent", options=options, jupyter=True)